# Predict Financial News Sentiment

### Environment
Ubuntu 22.04 LTS which includes **Python 3.9.12** and utilities *curl*, *git*, *vim*, *unzip*, *wget*, and *zip*. There is no *GPU* support.

The IPython Kernel allows you to execute Python code in the Notebook cell and Python console.

### Installing packages
- Run `!mamba list "package_name"` command to check the package installation status. For example,

```python
!mamba list numpy
"""
# packages in environment at /opt/conda:
#
# Name                    Version                   Build  Channel
numpy                     1.21.6           py39h18676bf_0    conda-forge
"""
```

    You can also try importing the package.

- Run the `!mamba install "package_name"` to install a package

### Excluding large files
HackerRank rejects any submission larger than **20MB**. Therefore, you must exclude any large files by adding these to the *.gitignore* file.
You can **Submit** code to validate the status of your submission.

In [7]:
# Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

pd.set_option("display.max_columns", 101)
pd.set_option("display.max_colwidth", None)

## Data Description

| Column    | Description                                         |
|:----------|:----------------------------------------------------|
| `news`  | news headline                                |
| `label`   | news sentiment - negative (`0`), neutral (`1`), positive (`2`) |

In [8]:
# The training dataset is already loaded below.
data = pd.read_csv("train.csv")
data.head()

,news,label
0,Residents access to the block is planned to be from Aleksandri Street .,1
1,"The company operates its business through two reportable segments , including Banking and Investment Services , and Non-Life Insurance .",1
2,"However , this increases signaling traffic which wastes network resources and allows fewer smartphones to connect .",0
3,"Ahlstrom Corporation STOCK EXCHANGE RELEASE 13.02.2008 Ahlstrom , a global leader in high performance fiber-based materials , today announced the completion of the acquisition of Friend Group Inc. , which consists of West Carrollton Parchment Company and West Carrollton Converting Company .",1
4,The winners included the Honda Odyssey for minivan and the Nissan Armada for large SUV .,1


## Deep Learning

Build a neural network that can predict the news sentiment.
- **The model's performance will be evaluated on the basis of weighted F1 score.**

> #### Task:
- **Submit the predictions on the test dataset using your optimized model** <br/>
    For each record in the test set (test.csv), predict the value of the `label` variable.  You should submit a CSV file with a header row and one row per test entry.

The file (`submissions.csv`) should have exactly 2 columns:

| Column    | Description                                         |
|:----------|:----------------------------------------------------|
| `news`  | news headline                                |
| `label`   | news sentiment - negative (`0`), neutral (`1`), positive (`2`) |

In [9]:
# The testing dataset is already loaded below.
test = pd.read_csv("test.csv")
test.head()

,news
0,"The company was supposed to deliver machinery to a veneer mill in the Tomsk region , in Russia ."
1,UNC Charlotte would also deploy SSH Tectia Connector to enable secure application connectivity .
2,"In 2009 , Lee & Man had a combined annual production capacity of close to 4.5 million tonnes of paper and 300,000 tonnes of pulp ."
3,"`` That 's a very high figure on the European scale , '' Noop said , recalling however that this also includes beer bought by Finnish tourists ."
4,"In Finland , the corresponding service is Alma Media 's Etuovi.com , Finland 's most popular and best known nationwide online service for home and property sales ."


In [ ]:
# Submission
submission_df.to_csv("submissions.csv", index=False)